In [3]:
# Importar librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score
)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [6]:
# Importar datos desde GitHub
url = "https://raw.github.com/1007451013i/Proyecto-Soluciones-Analiticas/main/heart.csv"
df = pd.read_csv(url)
print(df.shape)
df.head()

(297, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0


In [10]:
# Definir columna objetivo
target_col = "condition"

# Definir variables predictoras y variable objetivo
X = df.drop(columns=[target_col])
y = df[target_col]

# Dividir datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape

((207, 13), (90, 13))

In [11]:
# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
# Realizar función para calcular métricas
def calcular_metricas(y_true, y_pred, y_proba=None):
    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
    }
    if y_proba is not None:
        metrics["roc_auc"] = roc_auc_score(y_true, y_proba)
    else:
        metrics["roc_auc"] = np.nan
    return metrics

In [13]:
# Modelo 1: Regresión Logística
lr = LogisticRegression(
    max_iter=1000,
    solver="liblinear",
    random_state=42
)

# Entrenar los datos escalados
lr.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_lr = lr.predict(X_test_scaled)
y_proba_lr = lr.predict_proba(X_test_scaled)[:, 1]

# Calcular métricas
metrics_lr = calcular_metricas(y_test, y_pred_lr, y_proba_lr)
metrics_lr

{'accuracy': 0.8777777777777778,
 'precision': 0.9428571428571428,
 'recall': 0.7857142857142857,
 'f1': 0.8571428571428571,
 'roc_auc': np.float64(0.9350198412698413)}

In [14]:
# Modelo 2: Random Forest
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=5,
    min_samples_split=4,
    min_samples_leaf=2,
    random_state=42
)

# Entrenar los datos sin escalar
rf.fit(X_train, y_train)

# Realizar predicciones
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

# Calcular métricas
metrics_rf = calcular_metricas(y_test, y_pred_rf, y_proba_rf)
metrics_rf

{'accuracy': 0.8444444444444444,
 'precision': 0.8888888888888888,
 'recall': 0.7619047619047619,
 'f1': 0.8205128205128205,
 'roc_auc': np.float64(0.9315476190476191)}

In [15]:
# Modelo 3: Gradient Boosting
gb = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

# Entrenar los datos sin escalar
gb.fit(X_train, y_train)

# Realizar predicciones
y_pred_gb = gb.predict(X_test)
y_proba_gb = gb.predict_proba(X_test)[:, 1]

# Calcular métricas
metrics_gb = calcular_metricas(y_test, y_pred_gb, y_proba_gb)
metrics_gb

{'accuracy': 0.8333333333333334,
 'precision': 0.8461538461538461,
 'recall': 0.7857142857142857,
 'f1': 0.8148148148148148,
 'roc_auc': np.float64(0.9117063492063492)}

In [16]:
# Comparar los 3 modelos
resultados = pd.DataFrame({
    "Modelo": ["Logistic Regression", "Random Forest", "Gradient Boosting"],
    "Accuracy": [metrics_lr["accuracy"], metrics_rf["accuracy"], metrics_gb["accuracy"]],
    "Precision": [metrics_lr["precision"], metrics_rf["precision"], metrics_gb["precision"]],
    "Recall": [metrics_lr["recall"], metrics_rf["recall"], metrics_gb["recall"]],
    "F1": [metrics_lr["f1"], metrics_rf["f1"], metrics_gb["f1"]],
    "ROC_AUC": [metrics_lr["roc_auc"], metrics_rf["roc_auc"], metrics_gb["roc_auc"]],
})

resultados.sort_values(by="ROC_AUC", ascending=False)

,Modelo,Accuracy,Precision,Recall,F1,ROC_AUC
0,Logistic Regression,0.877778,0.942857,0.785714,0.857143,0.935020
1,Random Forest,0.844444,0.888889,0.761905,0.820513,0.931548
2,Gradient Boosting,0.833333,0.846154,0.785714,0.814815,0.911706
